In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
###Group by

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-17 11:00:00|  41| 513.6600000000003|            46|
|2020-01-30 13:00:00|  75| 1330.859999999999|            76|
|2020-01-27 20:00:00| 244|398.00000000000006|            16|
|2020-01-16 10:00:00|  74|1021.2199999999987|            80|
|2020-01-24 17:00:00| 260| 308.0600000000001|            27|
|2020-01-03 01:00:00|  80|             80.61|             6|
|2020-01-08 14:00:00|  33| 687.7900000000001|            34|
|2020-01-07 19:00:00|  66|324.78000000000003|            19|
|2020-01-14 12:00:00|  21|             90.22|             2|
|2020-01-17 17:00:00|  41| 845.2499999999994|            66|
|2020-01-02 09:00:00|   7|367.50000000000017|            30|
|2020-01-15 20:00:00| 145|             39.41|             2|
|2020-01-06 19:00:00|  92| 293.2000000000001|            18|
|2020-01-24 11:00:00| 12

In [11]:
#the more partition the more cost of reshuffling
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [12]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [13]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [14]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [ ]:
### Join

In [15]:
#To reduce repeat process we read from pq file we save. Otherwise it will do the query process from the start
# In big dataset we will see the big different of resource usage
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [16]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [17]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [18]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [19]:
df_join = spark.read.parquet('data/report/revenue/total')

In [20]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [21]:
df_zones = spark.read.parquet('zones/')

In [22]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [23]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

In [24]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  10|              null|                null|             42.41|                    2|        10|   Queens|        Baisley Park|   Boro Zone|
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00| 